In [1]:
import os 

from AEMG.data_utils import DynamicsDataset
from AEMG.systems.utils import get_system

import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline

import torch 
from torch.utils.data import DataLoader
from AEMG.models import *

from tqdm.notebook import tqdm 
import pickle

In [2]:
system = get_system("pendulum")
# config_fname = "config/pendulum_lqr_1K.txt"
config_fname = "config/physics_pendulum.txt"

with open(config_fname, 'r') as f:
    config = eval(f.read())

dataset = DynamicsDataset(config)

100%|██████████| 1000/1000 [00:24<00:00, 41.27it/s]


In [3]:
# Split into train and test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=True)


In [4]:
encoder  = Encoder(config["high_dims"],config["low_dims"])
dynamics = LatentDynamics(config["low_dims"])
decoder  = Decoder(config["low_dims"],config["high_dims"])

In [5]:
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(set(list(encoder.parameters()) + list(dynamics.parameters()) + list(decoder.parameters())), 
    lr=config["learning_rate"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, threshold=0.001, patience=5, verbose=True)

In [6]:
train_losses = {'loss_ae1': [], 'loss_ae2': [], 'loss_dyn': [], 'loss_total': []}
test_losses = {'loss_ae1': [], 'loss_ae2': [], 'loss_dyn': [], 'loss_total': []}

patience = config["patience"]

for epoch in tqdm(range(config["epochs"])):
    loss_ae1_train = 0
    loss_ae2_train = 0
    loss_dyn_train = 0

    current_train_loss = 0
    epoch_train_loss = 0

    warmup = 1 if epoch <= config["warmup"] else 0

    encoder.train()
    dynamics.train()
    decoder.train()

    counter = 0
    for i, (x_t, x_tau) in enumerate(train_loader,0):
        optimizer.zero_grad()

        # Forward pass
        z_t = encoder(x_t)
        x_t_pred = decoder(z_t)

        z_tau_pred = dynamics(z_t)
        z_tau = encoder(x_tau)
        x_tau_pred = decoder(z_tau_pred)

        # Compute losses
        loss_ae1 = criterion(x_t, x_t_pred)
        loss_ae2 = criterion(x_tau, x_tau_pred)
        loss_dyn = criterion(z_tau, z_tau_pred)

        loss_total = loss_ae1 + loss_ae2 + warmup * loss_dyn

        # Backward pass
        loss_total.backward()
        optimizer.step()

        current_train_loss += loss_total.item()
        epoch_train_loss += loss_total.item()

        loss_ae1_train += loss_ae1.item()
        loss_ae2_train += loss_ae2.item()
        loss_dyn_train += loss_dyn.item() * warmup
        counter += 1

        if (i+1) % 100 == 0:
            print("Epoch: {}, Iteration: {}, Loss: {}".format(epoch, i+1, current_train_loss))
            current_train_loss = 0
        
    train_losses['loss_ae1'].append(loss_ae1_train / counter)
    train_losses['loss_ae2'].append(loss_ae2_train / counter)
    train_losses['loss_dyn'].append(loss_dyn_train / counter)
    train_losses['loss_total'].append(epoch_train_loss / counter)

    with torch.no_grad():
        loss_ae1_test = 0
        loss_ae2_test = 0
        loss_dyn_test = 0
        epoch_test_loss = 0

        encoder.eval()
        dynamics.eval()
        decoder.eval()

        counter = 0
        for i, (x_t, x_tau) in enumerate(test_loader,0):
            # Forward pass
            z_t = encoder(x_t)
            x_t_pred = decoder(z_t)

            z_tau_pred = dynamics(z_t)
            z_tau = encoder(x_tau)
            x_tau_pred = decoder(z_tau_pred)

            # Compute losses
            loss_ae1 = criterion(x_t, x_t_pred)
            loss_ae2 = criterion(x_tau, x_tau_pred)
            loss_dyn = criterion(z_tau, z_tau_pred)

            loss_total = loss_ae1 + loss_ae2 + warmup * loss_dyn

            epoch_test_loss += loss_total.item()

            loss_ae1_test += loss_ae1.item()
            loss_ae2_test += loss_ae2.item()
            loss_dyn_test += loss_dyn.item() * warmup
            counter += 1

        test_losses['loss_ae1'].append(loss_ae1_test / counter)
        test_losses['loss_ae2'].append(loss_ae2_test / counter)
        test_losses['loss_dyn'].append(loss_dyn_test / counter)
        test_losses['loss_total'].append(epoch_test_loss / counter)

        if epoch >= patience and np.mean(test_losses['loss_total'][-patience:]) >= np.mean(test_losses['loss_total'][-patience:-1]):
            break

        if epoch >= config["warmup"]:
            scheduler.step(epoch_test_loss / counter)
        
    print("Epoch: {}, Train Loss: {}, Test Loss: {}".format(epoch, epoch_train_loss / counter, epoch_test_loss / counter))

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0, Iteration: 100, Loss: 9.994495432823896
Epoch: 0, Iteration: 200, Loss: 2.9003128670156
Epoch: 0, Iteration: 300, Loss: 1.7275239853188396
Epoch: 0, Iteration: 400, Loss: 1.2929670689627528
Epoch: 0, Iteration: 500, Loss: 1.0083316192030907
Epoch: 0, Iteration: 600, Loss: 0.8789923274889588
Epoch: 0, Iteration: 700, Loss: 0.8711279924027622
Epoch: 0, Train Loss: 0.09743534713052213, Test Loss: 0.008019286694470794
Epoch: 1, Iteration: 100, Loss: 0.7187251537106931
Epoch: 1, Iteration: 200, Loss: 0.5815311335027218
Epoch: 1, Iteration: 300, Loss: 0.5069702477194369
Epoch: 1, Iteration: 400, Loss: 0.486048074439168
Epoch: 1, Iteration: 500, Loss: 0.4653272957075387
Epoch: 1, Iteration: 600, Loss: 0.44439219729974866
Epoch: 1, Iteration: 700, Loss: 0.430659027537331
Epoch: 1, Train Loss: 0.020214141178876162, Test Loss: 0.004042761436430738
Epoch: 2, Iteration: 100, Loss: 0.40772153274156153
Epoch: 2, Iteration: 200, Loss: 0.3895947812125087
Epoch: 2, Iteration: 300, Loss: 0.391

In [7]:
# Save the models
torch.save(encoder, os.path.join(config["model_dir"], "encoder.pt"))
torch.save(dynamics, os.path.join(config["model_dir"], "dynamics.pt"))
torch.save(decoder, os.path.join(config["model_dir"], "decoder.pt"))

In [8]:
# if log_dir doesn't exist, create it
if not os.path.exists(config["log_dir"]):
    os.makedirs(config["log_dir"])

# Save the losses as a pickle file
with open(os.path.join(config["log_dir"], "losses.pkl"), "wb") as f:
    pickle.dump({"train_losses": train_losses, "test_losses": test_losses}, f)